In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/churn-5000.csv')

In [ ]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)



num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)


result_df_op=result_df

X=result_df_op
y=churn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train=pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
#train=pd.concat([X, y], axis=1)
#test=pd.concat([X, y], axis=1)
X_train=X
X_test=X
y_train=y
y_test=y

In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0




opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=10
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=200
opt.enable_privacy=1

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios

array([0.85714286, 0.14285714])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/chk_p_GAN_MWOE_privacy_1_dataset5000_70-vs-30.pth')

Epoch : 1 Loss D real :  0.0265364113840476 Loss D fake :  0.011643865992094271 Loss G :  0.011713500249870681 Epsilon spent :  3.3807596723163544
Epoch : 2 Loss D real :  0.026599961922568496 Loss D fake :  0.01152580108871902 Loss G :  0.011698957308053396 Epsilon spent :  3.8832879783901513
Epoch : 3 Loss D real :  0.026726655742574587 Loss D fake :  0.011694465916501485 Loss G :  0.011788860915106916 Epsilon spent :  4.3858162844639486
Epoch : 4 Loss D real :  0.026604889371104914 Loss D fake :  0.011603598731948508 Loss G :  0.011500266443302584 Epsilon spent :  4.6538087602691025
Epoch : 5 Loss D real :  0.026768425802651114 Loss D fake :  0.011529741070403892 Loss G :  0.011671757806256695 Epsilon spent :  4.857850494922193
Epoch : 6 Loss D real :  0.02666811318438699 Loss D fake :  0.0117801593254305 Loss G :  0.011509690901666935 Epsilon spent :  5.061892229575283
Epoch : 7 Loss D real :  0.02671170102046378 Loss D fake :  0.011569857612390648 Loss G :  0.011479493462809513 Ep

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
    syn_data = model.generate(X_train.shape[0], class_ratios)
    X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

In [ ]:
X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv")
test.to_csv("drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataseat5000.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv")

Saved synthetic data at :  drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv


In [ ]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [ ]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.836946,0.000000,-0.469349,-0.469349,0.311793,0.000000,0.311793,0.836946,0.311793,-0.469349,-0.469349,0.000000,-0.469349,0.836946,-0.469349,0.000000,0.836946,0.000000,-0.469349,0.000000
1,0.836946,0.311793,2.468711,-0.469349,1.661072,-0.469349,0.311793,0.311793,0.311793,0.836946,0.311793,0.311793,-0.469349,0.311793,-0.469349,0.311793,-0.469349,0.311793,0.836946,0.836946
2,0.836946,-0.469349,-0.469349,0.000000,0.311793,-0.469349,0.311793,0.311793,-0.469349,0.000000,0.836946,-0.469349,0.311793,0.836946,0.311793,-0.469349,0.000000,0.000000,-0.469349,0.836946
3,0.836946,0.311793,-0.469349,0.836946,0.311793,1.661072,0.311793,0.311793,-0.469349,0.836946,0.836946,0.311793,0.311793,0.000000,1.267627,0.311793,0.000000,-0.469349,-0.469349,-0.469349
4,0.836946,0.311793,-0.469349,0.311793,0.836946,0.311793,0.836946,-0.469349,-0.469349,0.836946,0.311793,0.311793,0.311793,1.267627,-0.469349,0.836946,0.311793,-0.469349,0.836946,0.836946


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn_df, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_syn_df, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2768 195 83 454
pod:  0.845437616387337
pof:  0.06581167735403308
AUC:  0.889812969516652
accuracy:  0.9205714285714286


In [ ]:
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataseat5000.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.358999,0.039822,0.031013,0.200916,1.810602,-0.605655,1.410377,0.973277,0.272236,0.973277,-0.358999,-0.928147,0.000000,0.441866,0.242130,0.441866,-0.276281,-0.203610,-0.308443,0.201986
1,1.410377,-0.111723,-0.018455,0.200916,-0.462942,0.221536,0.000000,0.000000,-0.643134,0.000000,0.000000,-0.864143,-0.557091,0.326591,0.242130,0.326591,-0.249514,-0.077157,-0.308443,0.201986
2,-0.358999,-0.358999,0.004923,0.000000,-0.358999,-0.647981,0.000000,-0.358999,0.409120,-0.358999,-0.358999,0.172797,0.622058,0.289240,-0.284555,0.289240,-0.249514,0.480250,-0.308443,-0.872765
3,0.000000,0.900476,-0.018455,0.200916,-0.242985,0.517807,0.323957,-0.242985,0.683962,0.000000,0.000000,0.441866,0.323957,0.000000,0.242130,0.000000,-0.249514,-0.214692,-0.308443,0.201986
4,-0.358999,0.323957,-0.018455,0.200916,-0.358999,-0.013731,0.000000,-0.242985,0.000000,-0.557091,0.000000,-0.561457,-0.358999,0.000000,-0.284555,0.000000,-0.249514,-0.688589,-0.308443,0.201986


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1271 22 85 122
pod:  0.5893719806763285
pof:  0.017014694508894045
AUC:  0.7861786430837172
recall:  0.5893719806763285
precision:  0.8472222222222222
F1-Score:  0.6951566951566951
accuracy:  0.9286666666666666


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1286 7 104 103
pod:  0.4975845410628019
pof:  0.005413766434648105
AUC:  0.7460853873140769
recall:  0.4975845410628019
precision:  0.9363636363636364
F1-Score:  0.6498422712933754
accuracy:  0.926


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 199 8
pod:  0.03864734299516908
pof:  0.0007733952049497294
AUC:  0.5189369738951096
recall:  0.03864734299516908
precision:  0.8888888888888888
F1-Score:  0.07407407407407408
accuracy:  0.8666666666666667


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 196 11
pod:  0.05314009661835749
pof:  0.0007733952049497294
AUC:  0.5261833507067039
recall:  0.05314009661835749
precision:  0.9166666666666666
F1-Score:  0.10045662100456622
accuracy:  0.8686666666666667


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1268 25 156 51
pod:  0.2463768115942029
pof:  0.019334880123743233
AUC:  0.6135209657352299
recall:  0.2463768115942029
precision:  0.6710526315789473
F1-Score:  0.36042402826855124
accuracy:  0.8793333333333333


In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 167 40
pod:  0.1932367149758454
pof:  0.0007733952049497294
AUC:  0.5962316598854478
recall:  0.1932367149758454
precision:  0.975609756097561
F1-Score:  0.3225806451612903
accuracy:  0.888


In [ ]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 169 38
pod:  0.18357487922705315
pof:  0.0007733952049497294
AUC:  0.5914007420110516
recall:  0.18357487922705315
precision:  0.9743589743589743
F1-Score:  0.3089430894308943
accuracy:  0.8866666666666667


In [ ]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
88/88 [==============================] - 4s 8ms/step - loss: 0.3746 - accuracy: 0.8664 - val_loss: 0.2360 - val_accuracy: 0.8914
Epoch 2/30
88/88 [==============================] - 0s 5ms/step - loss: 0.2436 - accuracy: 0.9032 - val_loss: 0.1802 - val_accuracy: 0.9257
Epoch 3/30
88/88 [==============================] - 0s 5ms/step - loss: 0.2081 - accuracy: 0.9175 - val_loss: 0.1692 - val_accuracy: 0.9329
Epoch 4/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1853 - accuracy: 0.9261 - val_loss: 0.1426 - val_accuracy: 0.9500
Epoch 5/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1674 - accuracy: 0.9318 - val_loss: 0.1586 - val_accuracy: 0.9471
Epoch 6/30
88/88 [==============================] - 1s 7ms/step - loss: 0.1605 - accuracy: 0.9368 - val_loss: 0.1376 - val_accuracy: 0.9529
Epoch 7/30
88/88 [==============================] - 1s 8ms/step - loss: 0.1562 - accuracy: 0.9375 - val_loss: 0.1361 - val_accuracy: 0.9486
Epoch 8/30
88/88 [==

In [1]:
#RNN
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
import numpy as np
max_features = 10000 # number of words to consider as features
#create model 
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_encoded, y_syn_df, epochs=5, batch_size=128, validation_split=0.2)
y2_pred = model.predict(X_encoded_test)
y22_pred=y2_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

NameError: name 'X_encoded' is not defined